In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

class Model_Trigger_Classification(nn.Module):
  def __init__(self,dim=WL_trigger):
    super().__init__()
    self.fc1 = nn.Linear(WL_trigger,model_NP_layer1)
    self.fc2 = nn.Linear(model_NP_layer1,model_NP_layer2)
    self.fc3 = nn.Linear(model_NP_layer2,model_NP_layer3)
    self.fc4 = nn.Linear(model_NP_layer3,model_NP_layer4)
    self.fc5 = nn.Linear(model_NP_layer4,model_NP_layer5)
  def forward(self,x):

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)
    x = self.fc5(x)

    return x

def train_Model_Trigger_Classification(model,train_data,train_labels,val_data,val_labels,learning_rate,batch_size,epochs, patience=15, factor=0.2):

  flag_start_again = 0
  while(flag_start_again != 2):
    flag_start_again = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

    losses, epoch_lst, train_accs, val_accs  = [], [] ,[], []
    length_train = np.shape(train_data)[0]
    length_val = np.shape(val_data)[0]

    best_val_acc = 0
    tj = 0
    flagy = 0
    for j in range(epochs):
        if flagy == 1 or flag_start_again == 1:
          break

        perm = np.random.permutation(length_train)
        train_data = train_data[perm]
        train_labels = train_labels[perm]

        for i in range(0, length_train, batch_size):
            if (i + batch_size) > length_train:
                break
            y_in = train_data[i:i+batch_size,:]
            labels_in = train_labels[i:i+batch_size]
            y_in = torch.Tensor(y_in)
            labels_in = torch.Tensor(labels_in)
            labels_in = labels_in.long()

            output = model(y_in)
            loss = criterion(output, labels_in)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % int(batch_size*200) == 0:
              losses.append(float(loss)/batch_size)  # compute *average* loss
              epoch_lst.append(tj)
              train_cost = float(loss.detach().numpy())
              train_acc = estimate_accuracy_Model_Trigger_Classification(model, train_data, train_labels)
              val_acc = estimate_accuracy_Model_Trigger_Classification(model, val_data, val_labels)
              train_accs.append(train_acc)
              val_accs.append(val_acc)
              print("Epoch %d. [Val Acc %.2f%% ] [Train Acc %.2f%%, Loss %f]" % (
                    tj, val_acc, train_acc, train_cost))
              tj += 1

              scheduler.step(val_acc)
              for param_group in optimizer.param_groups:
                  curr_lr = param_group['lr']
              if curr_lr == learning_rate*np.power(factor,3):
                  print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
                  flagy = 1
                  flag_start_again = 2
                  break

              if tj == 10 and val_acc < 51:
                flag_start_again = 1

  del y_in
  del labels_in
  return losses, epoch_lst, train_accs, val_accs

def estimate_accuracy_Model_Trigger_Classification(model, data, labels):
    data = torch.Tensor(data)
    out = model(data)
    out = out.detach().numpy()
    pred = np.argmax(out, axis=1)
    num_errors = np.sum(pred != labels)
    num_real = np.sum(labels)

    accuracy = 100 * ((num_real - num_errors) / num_real)
    return accuracy

def run_Model_Trigger_Classification(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1, path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_Trigger_Classification()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_Trigger_Classification(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)
  test_accuracy = estimate_accuracy_Model_Trigger_Classification(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,learning_rate,batch_size,epochs]))
  return

class Model_Arrival_Times_3(nn.Module):
  def __init__(self,dim=(WL_trigger)):
    super().__init__()
    self.fc1 = nn.Linear(WL_trigger,model_NP_layer1)
    self.fc2 = nn.Linear(model_NP_layer1,model_NP_layer2)
    self.fc3 = nn.Linear(model_NP_layer2,model_NP_layer3)
    self.fc4 = nn.Linear(model_NP_layer3,model_NP_layer4)
    self.fc5 = nn.Linear(model_NP_layer4,model_P3_layer5)

  def forward(self,x):

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)
    x = self.fc5(x)
    x = F.sigmoid(x)
    x = WL_trigger_middle*x

    return x

def train_Model_Arrival_Times_3(model,train_data,train_labels,val_data,val_labels,learning_rate,batch_size,epochs, patience=15, factor=0.2):

  flag_start_again = 0
  while(flag_start_again != 2):
    flag_start_again = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.L1Loss()
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

    losses, epoch_lst, train_accs, val_accs  = [], [] ,[], []
    length_train = np.shape(train_data)[0]
    length_val = np.shape(val_data)[0]

    best_val_acc = 0
    flagy = 0
    for j in range(epochs):
        if flagy == 1 or flag_start_again == 1:
          break

        perm = np.random.permutation(length_train)
        train_data = train_data[perm]
        train_labels = train_labels[perm]

        for i in range(0, length_train, batch_size):
            if (i + batch_size) > length_train:
                break
            y_in = train_data[i:i+batch_size,:]
            labels_in = train_labels[i:i+batch_size,:]
            y_in = torch.Tensor(y_in)
            labels_in = torch.Tensor(labels_in).long()
            labels_in = labels_in.float()

            output = model(y_in)
            loss = loss_func(output, labels_in)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        losses.append(float(loss)/batch_size)  # compute *average* loss
        epoch_lst.append(j)
        train_cost = float(loss.detach().numpy())
        train_dist = estimate_avg_dist_Model_Arrival_Times_3(model, train_data, train_labels)
        val_dist = estimate_avg_dist_Model_Arrival_Times_3(model, val_data, val_labels)
        train_acc = accuracy_of_Model_Arrival_Times_3(model, train_data, train_labels)
        val_acc = accuracy_of_Model_Arrival_Times_3(model, val_data, val_labels)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        print("Epoch %d. [Val Avg Dist %.2f%%] [Train Avg Dist %.2f%%] [Val Acc %.2f%% ] [Train Acc %.2f%%, Loss %f]" % (
        j, val_dist, train_dist, val_acc, train_acc, train_cost))

        scheduler.step(val_acc)
        for param_group in optimizer.param_groups:
            curr_lr = param_group['lr']
        if curr_lr == learning_rate*np.power(factor,3):
            flag_start_again = 2
            print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
            flagy = 1
            break

        print(val_acc)
        if j == 10 and val_acc < 30:
          flag_start_again = 1

  del y_in
  del labels_in
  return losses, epoch_lst, train_accs, val_accs

def accuracy_of_Model_Arrival_Times_3(model, data, labels):
    data = torch.Tensor(data)
    labels = torch.Tensor(labels)
    out = model(data)
    abs_diff = torch.abs(out - labels)
    success_mask = abs_diff < success_marg
    success = success_mask.sum().item()
    all = success_mask.numel()

    return 100 * success / all

def distance_histogram_3(model, data, labels):

  num = WL_trigger_middle
  distance_lst = np.zeros(num)
  count_lst = np.zeros(num)
  for i in range(np.shape(data)[0]):
    # get a batch of data
    y_in = data[i,:]
    labels_in = labels[i,:]
    y_in = torch.Tensor(y_in)
    out = model(y_in)
    distance_lst[int(labels_in[0])] += abs(out[0] - labels_in[0])
    count_lst[int(labels_in[0])] += 1
    distance_lst[int(labels_in[1])] += abs(out[1] - labels_in[1])
    count_lst[int(labels_in[1])] += 1
    distance_lst[int(labels_in[2])] += abs(out[2] - labels_in[2])
    count_lst[int(labels_in[2])] += 1
  for j in range(num):
    if count_lst[j] == 0:
      count_lst[j] = 1
  pyplot.figure()
  pyplot.scatter(np.arange(num),(distance_lst/count_lst))
  pyplot.show()


def estimate_avg_dist_Model_Arrival_Times_3(model, data, labels):
    data = torch.Tensor(data)
    labels = torch.Tensor(labels)
    out = model(data)
    abs_diff = torch.abs(out - labels)
    avg_dist = abs_diff.sum(dim=1).mean().item()

    return avg_dist

def run_Model_Arrival_Times_3(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1,path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_Arrival_Times_3()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_Arrival_Times_3(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)
  test_accuracy = accuracy_of_Model_Arrival_Times_3(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,learning_rate,batch_size,epochs]))
  return

class Model_Arrival_Times_2(nn.Module):
  def __init__(self,dim=(WL_trigger)):
    super().__init__()
    self.fc1 = nn.Linear(WL_trigger,model_NP_layer1)
    self.fc2 = nn.Linear(model_NP_layer1,model_NP_layer2)
    self.fc3 = nn.Linear(model_NP_layer2,model_NP_layer3)
    self.fc4 = nn.Linear(model_NP_layer3,model_NP_layer4)
    self.fc5 = nn.Linear(model_NP_layer4,model_P2_layer5)
    self.fc6 = nn.Linear(model_P2_layer5,model_P2_layer6)

  def forward(self,x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)
    x = self.fc5(x)
    x = F.relu(x)
    x = self.fc6(x)
    x = F.sigmoid(x)
    x = WL_trigger_middle*x

    return x

def train_Model_Arrival_Times_2(model,train_data,train_labels,val_data,val_labels,learning_rate,batch_size,epochs, patience=15, factor=0.2):
  flag_start_again = 0
  while(flag_start_again != 2):
    flag_start_again = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.L1Loss()
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

    losses, epoch_lst, train_accs, val_accs  = [], [] ,[], []
    length_train = np.shape(train_data)[0]
    length_val = np.shape(val_data)[0]

    best_val_acc = 0
    flagy = 0
    for j in range(epochs):
        if flagy == 1 or flag_start_again == 1:
          break
        perm = np.random.permutation(length_train)
        train_data = train_data[perm]
        train_labels = train_labels[perm]

        for i in range(0, length_train, batch_size):
            if (i + batch_size) > length_train:
                break
            y_in = train_data[i:i+batch_size,:]
            labels_in = train_labels[i:i+batch_size,:]
            y_in = torch.Tensor(y_in)
            labels_in = torch.Tensor(labels_in).long()
            labels_in = labels_in.float()

            output = model(y_in)
            loss = loss_func(output, labels_in)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        losses.append(float(loss)/batch_size)  # compute *average* loss
        epoch_lst.append(j)
        train_cost = float(loss.detach().numpy())
        train_dist = estimate_avg_dist_Model_Arrival_Times_2(model, train_data, train_labels)
        val_dist = estimate_avg_dist_Model_Arrival_Times_2(model, val_data, val_labels)
        train_acc = accuracy_of_Model_Arrival_Times_2(model, train_data, train_labels)
        val_acc = accuracy_of_Model_Arrival_Times_2(model, val_data, val_labels)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        print("Epoch %d. [Val Avg Dist %.2f%%] [Train Avg Dist %.2f%%] [Val Acc %.2f%% ] [Train Acc %.2f%%, Loss %f]" % (
        j, val_dist, train_dist, val_acc, train_acc, train_cost))

        scheduler.step(val_acc)
        for param_group in optimizer.param_groups:
            curr_lr = param_group['lr']
        if curr_lr == learning_rate*np.power(factor,3):
            print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
            flagy = 1
            flag_start_again = 2
            break

        if j == 10 and val_acc < 30:
          flag_start_again = 1

  del y_in
  del labels_in
  return losses, epoch_lst, train_accs, val_accs

def accuracy_of_Model_Arrival_Times_2(model, data, labels):
    data = torch.Tensor(data)
    labels = torch.Tensor(labels)
    out = model(data)
    abs_diff = torch.abs(out - labels)
    success_mask = abs_diff < success_marg
    success = success_mask.sum().item()
    all = success_mask.numel()

    return 100 * success / all

def distance_histogram_2(model, data, labels):

  num = WL_trigger_middle
  distance_lst = np.zeros(num)
  count_lst = np.zeros(num)
  for i in range(np.shape(data)[0]):
    # get a batch of data
    y_in = data[i,:]
    labels_in = labels[i,:]
    y_in = torch.Tensor(y_in)
    out = model(y_in)
    distance_lst[int(labels_in[0])] += abs(out[0] - labels_in[0])
    count_lst[int(labels_in[0])] += 1
    distance_lst[int(labels_in[1])] += abs(out[1] - labels_in[1])
    count_lst[int(labels_in[1])] += 1
  for j in range(num):
    if count_lst[j] == 0:
      count_lst[j] = 1
  pyplot.figure()
  pyplot.scatter(np.arange(num),(distance_lst/count_lst))
  pyplot.show()


def estimate_avg_dist_Model_Arrival_Times_2(model, data, labels):
    data = torch.Tensor(data)
    labels = torch.Tensor(labels)
    out = model(data)
    abs_diff = torch.abs(out - labels)
    avg_dist = abs_diff.sum(dim=1).mean().item()

    return avg_dist


def run_Model_Arrival_Times_2(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1,path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_Arrival_Times_2()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_Arrival_Times_2(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)
  test_accuracy = accuracy_of_Model_Arrival_Times_2(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,learning_rate,batch_size,epochs]))
  return

class Model_Arrival_Times_1(nn.Module):
  def __init__(self,dim=(WL_trigger)):
    super().__init__()
    self.fc1 = nn.Linear(WL_trigger,model_NP_layer1)
    self.fc2 = nn.Linear(model_NP_layer1,model_NP_layer2)
    self.fc3 = nn.Linear(model_NP_layer2,model_NP_layer3)
    self.fc4 = nn.Linear(model_NP_layer3,model_NP_layer4)
    self.fc5 = nn.Linear(model_NP_layer4,model_P1_layer5)
    self.fc6 = nn.Linear(model_P1_layer5,model_P1_layer6)

  def forward(self,x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)
    x = self.fc5(x)
    x = F.relu(x)
    x = self.fc6(x)
    x = F.sigmoid(x)
    x = WL_trigger_middle*x

    return x

def train_Model_Arrival_Times_1(model,train_data,train_labels,val_data,val_labels,learning_rate,batch_size,epochs, patience=15, factor=0.2):

  flag_start_again = 0
  while(flag_start_again != 2):
    flag_start_again = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.L1Loss()

    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

    losses, epoch_lst, train_accs, val_accs  = [], [] ,[], []
    length_train = np.shape(train_data)[0]
    length_val = np.shape(val_data)[0]

    best_val_acc = 0
    flagy = 0
    for j in range(epochs):
        if flagy == 1 or flag_start_again == 1:
          break

        perm = np.random.permutation(length_train)
        train_data = train_data[perm]
        train_labels = train_labels[perm]

        for i in range(0, length_train, batch_size):
            if (i + batch_size) > length_train:
                break
            y_in = train_data[i:i+batch_size,:]
            labels_in = train_labels[i:i+batch_size]
            y_in = torch.Tensor(y_in)
            labels_in = torch.Tensor(labels_in)
            labels_in = labels_in.float()
            labels_in = labels_in[:,None]

            output = model(y_in)
            loss = loss_func(output, labels_in)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        losses.append(float(loss)/batch_size)  # compute *average* loss
        epoch_lst.append(j)
        train_cost = float(loss.detach().numpy())
        train_dist = estimate_avg_dist_Model_Arrival_Times_1(model, train_data, train_labels)
        val_dist = estimate_avg_dist_Model_Arrival_Times_1(model, val_data, val_labels)
        train_acc = accuracy_of_Model_Arrival_Times_1(model, train_data, train_labels)
        val_acc = accuracy_of_Model_Arrival_Times_1(model, val_data, val_labels)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        print("Epoch %d. [Val Avg Dist %.2f%%] [Train Avg Dist %.2f%%] [Val Acc %.2f%% ] [Train Acc %.2f%%, Loss %f]" % (
        j, val_dist, train_dist, val_acc, train_acc, train_cost))
        scheduler.step(val_acc)
        for param_group in optimizer.param_groups:
            curr_lr = param_group['lr']
        if curr_lr == learning_rate*np.power(factor,3):
            print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
            flagy = 1
            flag_start_again = 2
            break

        if j == 10 and val_acc < 30:
          flag_start_again = 1

  del y_in
  del labels_in
  return losses, epoch_lst, train_accs, val_accs

def accuracy_of_Model_Arrival_Times_1(model, data, labels):
    data = torch.Tensor(data)
    labels = torch.Tensor(labels)
    labels = labels.float()
    labels = labels[:,None]
    out = model(data)
    abs_diff = torch.abs(out - labels)
    success_mask = abs_diff < success_marg
    success = success_mask.sum().item()
    all = success_mask.numel()

    return 100 * success / all

def distance_histogram_1(model, data, labels):

  num = WL_trigger_middle
  distance_lst = np.zeros(num)
  count_lst = np.zeros(num)
  for i in range(np.shape(data)[0]):
    # get a batch of data
    y_in = data[i,:]
    labels_in = labels[i]
    labels_in = labels_in.float()
    labels_in = labels_in[:,None]
    y_in = torch.Tensor(y_in)
    out = model(y_in)
    distance_lst[int(labels_in[0])] += abs(out[0] - labels_in[0])
    count_lst[int(labels_in[0])] += 1
  for j in range(num):
    if count_lst[j] == 0:
      count_lst[j] = 1
  pyplot.figure()
  pyplot.scatter(np.arange(num),(distance_lst/count_lst))
  pyplot.show()


def estimate_avg_dist_Model_Arrival_Times_1(model, data, labels):
    data = torch.Tensor(data)
    labels = torch.Tensor(labels)
    labels = labels.float()
    labels = labels[:,None]
    out = model(data)
    abs_diff = torch.abs(out - labels)
    avg_dist = abs_diff.sum(dim=1).mean().item()

    return avg_dist


def run_Model_Arrival_Times_1(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1,path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_Arrival_Times_1()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_Arrival_Times_1(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)
  test_accuracy = accuracy_of_Model_Arrival_Times_1(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,learning_rate,batch_size,epochs]))
  return